# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')


# Limpeza inicial
df = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)

# Tratar missing values em tempo_emprego
df['tempo_emprego'] = df['tempo_emprego'].fillna(0)

# Separar variáveis
X = df.drop('renda', axis=1)
y = df['renda']

# 2. Separar base treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# 3. Pré-processamento
numeric_features = ['qtd_filhos', 'idade', 'tempo_emprego', 'qt_pessoas_residencia']
categorical_features = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 
                       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

# 4. Regularização Ridge
print("=== REGULARIZAÇÃO RIDGE ===")
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_ridge_score = 0
best_ridge_alpha = 0

for alpha in alphas:
    ridge_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('ridge', Ridge(alpha=alpha))
    ])
    
    ridge_pipe.fit(X_train, y_train)
    y_pred = ridge_pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Alpha {alpha}: R² = {r2:.4f}")
    
    if r2 > best_ridge_score:
        best_ridge_score = r2
        best_ridge_alpha = alpha

print(f"\nMelhor Ridge: Alpha {best_ridge_alpha} com R² = {best_ridge_score:.4f}")

# 5. Regularização LASSO
print("\n=== REGULARIZAÇÃO LASSO ===")
best_lasso_score = 0
best_lasso_alpha = 0

for alpha in alphas:
    lasso_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('lasso', Lasso(alpha=alpha))
    ])
    
    lasso_pipe.fit(X_train, y_train)
    y_pred = lasso_pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Alpha {alpha}: R² = {r2:.4f}")
    
    if r2 > best_lasso_score:
        best_lasso_score = r2
        best_lasso_alpha = alpha

print(f"\nMelhor LASSO: Alpha {best_lasso_alpha} com R² = {best_lasso_score:.4f}")

# 6. Stepwise (usando RFE)
print("\n=== STEPWISE (RFE) ===")
lr_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('linear', LinearRegression())
])

# Ajustar primeiro para obter os features
lr_pipe.fit(X_train, y_train)

# RFE para seleção de features
rfe = RFE(estimator=LinearRegression(), n_features_to_select=10)
rfe_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('rfe', rfe),
    ('linear', LinearRegression())
])

rfe_pipe.fit(X_train, y_train)
y_pred_rfe = rfe_pipe.predict(X_test)
r2_rfe = r2_score(y_test, y_pred_rfe)
print(f"Stepwise (RFE) R² = {r2_rfe:.4f}")

# 7. Comparação inicial
print("\n=== COMPARAÇÃO INICIAL ===")
print(f"Ridge:  R² = {best_ridge_score:.4f}")
print(f"LASSO:  R² = {best_lasso_score:.4f}")
print(f"Stepwise: R² = {r2_rfe:.4f}")

# 8. Melhorias criativas
print("\n=== MELHORIAS CRIATIVAS ===")
# Criar novas features
df_enhanced = df.copy()
df_enhanced['idade_quadrado'] = df_enhanced['idade'] ** 2
df_enhanced['tempo_emprego_quadrado'] = df_enhanced['tempo_emprego'] ** 2
df_enhanced['idade_tempo_interacao'] = df_enhanced['idade'] * df_enhanced['tempo_emprego']
df_enhanced['renda_per_capita'] = df_enhanced['renda'] / df_enhanced['qt_pessoas_residencia']
df_enhanced['qt_pessoas_residencia'] = df_enhanced['qt_pessoas_residencia'].replace(0, 1)

X_enhanced = df_enhanced.drop('renda', axis=1)

# Separar novamente
X_train_enh, X_test_enh, y_train_enh, y_test_enh = train_test_split(
    X_enhanced, y, test_size=0.25, random_state=42
)

# Ajustar numeric features para versão melhorada
numeric_features_enh = numeric_features + ['idade_quadrado', 'tempo_emprego_quadrado', 
                                         'idade_tempo_interacao', 'renda_per_capita']

preprocessor_enh = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_enh),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ])

# Testar Ridge com features melhoradas
ridge_enh_pipe = Pipeline([
    ('preprocessor', preprocessor_enh),
    ('ridge', Ridge(alpha=best_ridge_alpha))
])

ridge_enh_pipe.fit(X_train_enh, y_train_enh)
y_pred_enh = ridge_enh_pipe.predict(X_test_enh)
r2_enh = r2_score(y_test_enh, y_pred_enh)
print(f"Ridge com features melhoradas: R² = {r2_enh:.4f}")

# 9. Árvore de Regressão
print("\n=== ÁRVORE DE REGRESSÃO ===")
tree_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeRegressor(random_state=42, max_depth=10))
])

tree_pipe.fit(X_train, y_train)
y_pred_tree = tree_pipe.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)
print(f"Árvore de Regressão: R² = {r2_tree:.4f}")

# 10. Comparação Final
print("\n" + "="*50)
print("COMPARAÇÃO FINAL DOS MODELOS")
print("="*50)
print(f"Ridge:           R² = {best_ridge_score:.4f}")
print(f"LASSO:           R² = {best_lasso_score:.4f}")
print(f"Stepwise:        R² = {r2_rfe:.4f}")
print(f"Ridge Melhorado: R² = {r2_enh:.4f}")
print(f"Árvore:          R² = {r2_tree:.4f}")

# Identificar melhor modelo
results = {
    'Ridge': best_ridge_score,
    'LASSO': best_lasso_score,
    'Stepwise': r2_rfe,
    'Ridge_Melhorado': r2_enh,
    'Arvore': r2_tree
}

best_model = max(results, key=results.get)
print(f"\n🎯 MELHOR MODELO: {best_model} com R² = {results[best_model]:.4f}")

=== REGULARIZAÇÃO RIDGE ===
Alpha 0: R² = 0.2685
Alpha 0.001: R² = 0.2685
Alpha 0.005: R² = 0.2685
Alpha 0.01: R² = 0.2685
Alpha 0.05: R² = 0.2685
Alpha 0.1: R² = 0.2685

Melhor Ridge: Alpha 0.1 com R² = 0.2685

=== REGULARIZAÇÃO LASSO ===
Alpha 0: R² = 0.2685
Alpha 0.001: R² = 0.2685
Alpha 0.005: R² = 0.2685
Alpha 0.01: R² = 0.2685
Alpha 0.05: R² = 0.2685
Alpha 0.1: R² = 0.2685

Melhor LASSO: Alpha 0.1 com R² = 0.2685

=== STEPWISE (RFE) ===
Stepwise (RFE) R² = 0.2637

=== COMPARAÇÃO INICIAL ===
Ridge:  R² = 0.2685
LASSO:  R² = 0.2685
Stepwise: R² = 0.2637

=== MELHORIAS CRIATIVAS ===
Ridge com features melhoradas: R² = 0.8558

=== ÁRVORE DE REGRESSÃO ===
Árvore de Regressão: R² = 0.3305

COMPARAÇÃO FINAL DOS MODELOS
Ridge:           R² = 0.2685
LASSO:           R² = 0.2685
Stepwise:        R² = 0.2637
Ridge Melhorado: R² = 0.8558
Árvore:          R² = 0.3305

🎯 MELHOR MODELO: Ridge_Melhorado com R² = 0.8558
